In [73]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import warnings
warnings.filterwarnings("ignore")

np.random.seed(132)

In [74]:
labels = pd.read_csv('../data/Labels.csv')
data = pd.read_csv('../data/Data.csv')

In [75]:
X_train = data.to_numpy()
X_train.shape

(13611, 17)

In [76]:
labels = labels.drop(columns=['Unnamed: 0'])
data = data.drop(columns=['Unnamed: 0'])

In [77]:
enc = OneHotEncoder()
ohe_labels = enc.fit_transform(labels)

In [78]:
y_train = ohe_labels.toarray()
y_train.shape

(13611, 7)

In [79]:
def sigmoid(x):
    #input is a matrix. Return element wise sigmoid
    return 1/(1+np.exp(-x))

def softmax(x):
    return np.exp(x)/(np.sum(np.exp(x)))

In [80]:
class NeuralNetworkClassifier():

    def __init__(self, layer_weights=[32,64]):
        self.layers = layer_weights
        self.output_layer = None
        self.input_layer = None

        #Weights and biases are to be stored as KV pairs
        self.W = {}
        self.B = {}

        #Intermediate computations to be stored as KV pairs
        self.A = {}
        self.H = {}


    '''
    Mandatory call for fitting training data on the model.
    Required step since input and output layer size are unknown.
    '''
    def fit(self, X, Y):

        self.input_layer = X.shape[1] #num features
        self.output_layer = Y.shape[1] #num classes

        self.layers = [self.input_layer] + self.layers + [self.output_layer]

        print(f"Neural network layer sizes : {self.layers}")
              
        #initialize weights for the layers
        self.initialize_weights()

        #Call forward propagation to train model
        preds = []
        for x,y in zip(X,Y):
            self.forward_propagation(x)
            preds.append(self.H[(len(self.layers)-1)])

        return preds

    '''
    Initialize the weights and biases once the layer sizes are known
    '''
    def initialize_weights(self):

        for layer_num in range(1,len(self.layers)):
            self.W[layer_num] = np.random.randn(self.layers[layer_num-1], self.layers[layer_num])
            print(f"Shape of W{layer_num} = {self.W[layer_num].shape }")
            self.B[layer_num] = np.random.randn(self.layers[layer_num])
            print(f"Shape of B{layer_num} = {self.B[layer_num].shape }")


    
    def forward_propagation(self, X):

        #x is a single datapoint (num_feats, )
        
        #Check if num features are accurate
        if X.shape[0] != self.input_layer:
            print(f"Invalid shape. {X.shape[1]} does not match {self.input_layer}")
            return
        
        #reshaped to have 1 row and all elements as column values (hence -1)
        self.H[0] = X.reshape(1,-1)

        #Repeat from layer 1 till layer l-1 (for final layer the output activation will change, so that will be done separately)
        #For [i/p, l1, l2, o/p] sequence of layers, this loop runs for l1 and l2
        for layer in range(1, len(self.layers)-1):
            self.A[layer] = np.matmul(self.H[(layer-1)],self.W[layer]) + self.B[(layer)]
            self.H[layer] = sigmoid(self.A[layer]) #compute sigmoid or orher activation here
        
        #Reached output layer. Calculate A[final layer] followed by H[final layer]
        self.A[(len(self.layers)-1)] = np.matmul(self.H[(len(self.layers)-2)],self.W[(len(self.layers)-1)]) + self.B[(len(self.layers)-1)]
        self.H[(len(self.layers)-1)] = softmax(self.A[(len(self.layers)-1)])

        return 


In [81]:
nn = NeuralNetworkClassifier([32, 64])
sample_preds = nn.fit(X_train, y_train)

Neural network layer sizes : [17, 32, 64, 7]
Shape of W1 = (17, 32)
Shape of B1 = (32,)
Shape of W2 = (32, 64)
Shape of B2 = (64,)
Shape of W3 = (64, 7)
Shape of B3 = (7,)


In [88]:
sample_preds[0:5]

[array([[7.80431311e-05, 9.99915224e-01, 1.42138979e-08, 1.54798162e-06,
         2.87227341e-06, 1.81234640e-10, 2.29867598e-06]]),
 array([[7.80431311e-05, 9.99915224e-01, 1.42138979e-08, 1.54798162e-06,
         2.87227341e-06, 1.81234640e-10, 2.29867598e-06]]),
 array([[7.80431311e-05, 9.99915224e-01, 1.42138979e-08, 1.54798162e-06,
         2.87227341e-06, 1.81234640e-10, 2.29867598e-06]]),
 array([[7.80431311e-05, 9.99915224e-01, 1.42138979e-08, 1.54798162e-06,
         2.87227341e-06, 1.81234640e-10, 2.29867598e-06]]),
 array([[7.80431311e-05, 9.99915224e-01, 1.42138979e-08, 1.54798162e-06,
         2.87227341e-06, 1.81234640e-10, 2.29867598e-06]])]

In [87]:
#Check if softmax works as expected (each row sums to 1)
np.sum(sample_preds)/len(sample_preds)

1.0000000000000002